Este Notebook tiene el objetivo de realizar un analisis del historico de solicitudes de compras para definir una reglas de reabastecimiento o recompra de insumos basado en la recurrencia y el volumen

In [207]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

# Configurar renderer para VS Code notebooks
pio.renderers.default = "notebook_connected"

In [208]:
df_compras = pd.read_excel('PRODUCCION.xlsx', sheet_name='COMPRAS', usecols='A:AE', skiprows=1)

In [209]:
print(df_compras.columns.tolist())

['ESTADO', 'CLIENTE', 'OP', 'OP DETALLE', 'PROVEEDOR', 'INSUMO', 'COLOR', 'CANT. elemento', 'UNIDAD', 'ELABORACIÓN DE PRODUCTO', 'CANTIDADES PRODUCTO', 'ID COSTEO', 'DETALLE', 'FECHA RECIBIDO', 'ES TELA', 'CONSUMO', 'COSTO SIN IVA', 'CONSUMO REAL', 'COSTO UNITARIO', 'PRODUCTO COMERCIALIZADO', 'FECHA SOLICITUD', 'FECHA PROVEEDOR', 'FECHA PRODUCCIÓN', 'NOTA', '_id', '_id_costeo', 'URL DETALLE', 'URL COSTEO', '_OPDETALLE', '_fecha promesa', 'MUESTAS ']


## Limpieza y normalización

In [210]:
df_compras['CANT. elemento'] = df_compras['CANTIDADES PRODUCTO'] * df_compras['CONSUMO']

In [211]:
# limpiar N/A en ESTADO
df_compras = df_compras.dropna(subset=["ESTADO", "INSUMO"])
# Limpiar materias primas
df_compras = df_compras[~(df_compras["ES TELA"] == True)]

# normalizar columna INSUMO eliminando tildes y aplicando mayusculas y reemplazar multiples espacios por uno solo
df_compras["INSUMO"] = (
    df_compras["INSUMO"]
    .str.normalize("NFKD")
    .str.encode("ascii", errors="ignore")
    .str.decode("utf-8")
    .str.upper()
    .str.strip()
    .str.replace(r"\s+", " ", regex=True)
)
# LIMPIAR ELEMENTOS NO INSUMOS SI CONTIENE EL SUBSTRING
elementos_no_insumos = [
    "PAQUETE COMPLETO",
    "TRANSPORTE",
    "CORTE",
    "CONFECCION",
    "BORDADO",
    "ESTAMPADO",
    "SUBLIMACION",
    "OTRO",
    "SACO",
    "HODDIE",
    "REATA",
    "JEAN",
    "CAMISETA",
    "GORRA",
    "MALETA",
    "TOALLA",
    "OXFORD",
    "RIB",
    "SUPER VERTIGO",
    "DRIL",
    "TEJIDO",
    "EMPAQUE"
]
for elemento in elementos_no_insumos:
    df_compras = df_compras[~df_compras["INSUMO"].str.contains(elemento, na=False)]

# eliminar COLUMNAS INNECESARIAS (conservamos CONSUMO para normalización)
del_cols = [
    "CLIENTE",
    "OP",
    "OP DETALLE",
    "PROVEEDOR",
    "COLOR",
    "ELABORACIÓN DE PRODUCTO",
    "ID COSTEO",
    "DETALLE",
    "ES TELA",
    "CONSUMO REAL",
    "COSTO UNITARIO",
    "PRODUCTO COMERCIALIZADO"
]

# Conservar columnas necesarias en lugar de eliminar las últimas 8
cols_a_conservar = ['INSUMO', 'UNIDAD', 'FECHA RECIBIDO', 'CANT. elemento', 'CONSUMO', 'CANTIDADES PRODUCTO']
df_compras = df_compras[[c for c in cols_a_conservar if c in df_compras.columns]]
df_compras['UNIDAD'].fillna('Unidad', inplace=True)

# unificar CUELLO y CUELLOS en INSUMO
df_compras["INSUMO"] = df_compras["INSUMO"].replace({"CUELLOS": "CUELLO"})

## Agrupación por semanas

In [212]:
# crear columna basada en FECHA RECIBIDO asignando el numero de semana del año
df_compras['SEMANA AÑO'] = df_compras['FECHA RECIBIDO'].dt.isocalendar().week
# agrupar por SEMANA
df_insumos_semana = df_compras.groupby(['INSUMO', 'SEMANA AÑO'], as_index=False)['CANT. elemento'].sum()
# matriz de tiempo
df_matriz = df_insumos_semana.pivot(index='INSUMO', columns='SEMANA AÑO', values='CANT. elemento').fillna(0)

## Estadisticas

In [213]:
df_calc = pd.DataFrame(index=df_matriz.index)

# Número total de semanas en el periodo de análisis
TOTAL_SEMANAS = len(df_matriz.columns)

# estadisticas basicas (UNIDADES REALES para el ROP)
df_calc['CONSUMO_TOTAL'] = df_matriz.sum(axis=1)
df_calc['PROMEDIO_SEM'] = df_matriz.mean(axis=1)
df_calc['DESV_STD'] = df_matriz.std(axis=1)

# coeficiente de variacion
df_calc['CV'] = np.where(df_calc['PROMEDIO_SEM'] == 0, 0, df_calc['DESV_STD'] / df_calc['PROMEDIO_SEM'])

# Frecuencia de Movimiento
df_calc['PCT_SEMANAS_CONSUMO'] = (df_matriz > 0).sum(axis=1) / TOTAL_SEMANAS

# ═══════════════════════════════════════════════════════════════════════════════
# NUEVA LÓGICA MEJORADA: Agrupación por Familias y Normalización Logarítmica
# ═══════════════════════════════════════════════════════════════════════════════

def inferir_familia(nombre_insumo):
    """Clasifica el insumo en una familia lógica basada en palabras clave."""
    nombre = str(nombre_insumo).upper()
    
    if any(x in nombre for x in ['CREMALLERA', 'SLIDER', 'CRN', 'CRI', 'CRM', 'CRP', 'SLC']):
        return 'CREMALLERAS Y CIERRES'
    elif any(x in nombre for x in ['BOTON', 'BROCHE', 'HEBILLA', 'OJALETE', 'ANILLO', 'BOE', 'BRP', 'BRM', 'HEM']):
        return 'BOTONES Y HERRAJES'
    elif any(x in nombre for x in ['SESGO', 'CINTA', 'HILADILLO', 'CORDON', 'EMBONE', 'VELCRO', 'REATA', 'TIRA', 'SEL', 'CIR', 'COE', 'EMC', 'VES']):
        return 'CINTAS Y CORDONES'
    elif any(x in nombre for x in ['RESORTE', 'CAUCHO', 'REC', 'HIC']):
        return 'ELASTICOS'
    elif any(x in nombre for x in ['MARQUILLA', 'MAP', 'MAN']):
        return 'ETIQUETAS'
    elif any(x in nombre for x in ['CUELLO', 'PUNO', 'FAJON', 'CUP']):
        return 'TEJIDOS (CUELLOS/PUÑOS)'
    elif any(x in nombre for x in ['GUATA', 'GABARDINA', 'INDIGO', 'TELA', 'GUA', 'TIN']):
        return 'TEXTILES Y GUATAS'
    elif any(x in nombre for x in ['TANCA', 'PUNTERA', 'CORBATA', 'TAP']):
        return 'ACCESORIOS PLASTICOS'
    else:
        return 'OTROS INSUMOS'

# 1. Asignar Familia
df_calc['FAMILIA_LOGICA'] = df_calc.index.map(inferir_familia)

# 2. Transformación Logarítmica (Suaviza diferencias extremas de escala)
#    Usamos log1p para manejar log(0) de forma segura
df_calc['LOG_CONSUMO'] = np.log1p(df_calc['CONSUMO_TOTAL'])

# 3. Calcular Max y Min del LOG dentro de cada familia
df_calc['MAX_LOG_FAM'] = df_calc.groupby('FAMILIA_LOGICA')['LOG_CONSUMO'].transform('max')
df_calc['MIN_LOG_FAM'] = df_calc.groupby('FAMILIA_LOGICA')['LOG_CONSUMO'].transform('min')

# 4. Calcular Score Normalizado (Min-Max Scaling sobre Logaritmos)
#    Fórmula: (Valor - Min) / (Max - Min) -> Resultado entre 0 y 1
#    Manejamos el caso donde Max == Min (familia de 1 solo item o consumos idénticos)
df_calc['SCORE_INTRA_FAMILIA'] = np.where(
    df_calc['MAX_LOG_FAM'] == df_calc['MIN_LOG_FAM'],
    1.0, # Si es único, es el líder por defecto
    (df_calc['LOG_CONSUMO'] - df_calc['MIN_LOG_FAM']) / (df_calc['MAX_LOG_FAM'] - df_calc['MIN_LOG_FAM'])
)

# 5. Usar este Score para el Pareto (Multiplicado por 1000 para visualización)
df_calc['CONSUMO_NORMALIZADO'] = df_calc['SCORE_INTRA_FAMILIA'] * 1000

print(f"📊 Normalización Logarítmica por Familias Aplicada:")
print(f"   Se han clasificado {len(df_calc)} insumos en {df_calc['FAMILIA_LOGICA'].nunique()} familias.")

📊 Normalización Logarítmica por Familias Aplicada:
   Se han clasificado 97 insumos en 9 familias.


In [214]:
df_compras[df_compras['INSUMO'] == 'SEP0029-SESGO POLIESTER 8001']['CONSUMO'].describe()

count    32.000000
mean      1.290313
std       1.566079
min       0.080000
25%       0.080000
50%       0.080000
75%       2.837500
max       4.100000
Name: CONSUMO, dtype: float64

In [215]:
df_consumo_tipico['SEP0029-SESGO POLIESTER 8001']

np.float64(0.08)

In [216]:
df_calc

,CONSUMO_TOTAL,PROMEDIO_SEM,DESV_STD,CV,PCT_SEMANAS_CONSUMO,FAMILIA_LOGICA,LOG_CONSUMO,MAX_LOG_FAM,MIN_LOG_FAM,SCORE_INTRA_FAMILIA,CONSUMO_NORMALIZADO
INSUMO,,,,,,,,,,,
ANILLO EN D 3 CM,320.00,11.851852,61.584029,5.196152,0.037037,BOTONES Y HERRAJES,5.771441,12.485124,3.218876,0.275469,275.469132
BOE0028 BOTON ESTANDAR,264374.00,9791.629630,11904.651771,1.215799,1.000000,BOTONES Y HERRAJES,12.485124,12.485124,3.218876,1.000000,1000.000000
BOE0028 BOTON ESTANDAR 18L 4H,1211.00,44.851852,192.098273,4.282951,0.074074,BOTONES Y HERRAJES,7.100027,12.485124,3.218876,0.418848,418.848205
BOE0028 BOTON ESTANDAR 24L 4H,24.00,0.888889,4.618802,5.196152,0.037037,BOTONES Y HERRAJES,3.218876,12.485124,3.218876,0.000000,0.000000
BOTON 18L 4H,357.00,13.222222,59.828173,4.524820,0.074074,BOTONES Y HERRAJES,5.880533,12.485124,3.218876,0.287242,287.242168
...,...,...,...,...,...,...,...,...,...,...,...
TAP0034 TANCA PUNTERA,555.00,20.555556,51.995809,2.529526,0.370370,ACCESORIOS PLASTICOS,6.320768,6.320768,3.258097,1.000000,1000.000000
TIN0138-INDIGO 12 ONZ,343.10,12.707407,57.692382,4.540059,0.148148,TEXTILES Y GUATAS,5.840932,7.450196,2.580217,0.669554,669.554319
TIRA SOLO PUNOS 6 CM ANCHO,776.00,28.740741,149.341270,5.196152,0.037037,CINTAS Y CORDONES,6.655440,9.904482,1.472472,0.614678,614.677710


## Calculo lead times

In [217]:
# Definición de Lead Times (Tiempos de Entrega)
# Regla: 
# - Tejidos (Cuellos/Puños) requieren tejido y teñido: 15 días
# - Insumos estándar (Botones, Cremalleras, Cintas): 5 días (Stock local o rápido)

DIAS_TEJIDOS = 15
DIAS_ESTANDAR = 5

# Usamos la Familia Lógica para asignar el Lead Time
df_calc['LT_SEMANAS'] = np.where(
    df_calc['FAMILIA_LOGICA'] == 'TEJIDOS (CUELLOS/PUÑOS)', 
    DIAS_TEJIDOS, 
    DIAS_ESTANDAR
) / 7

print("✅ Lead Times calculados:")
print(df_calc.groupby('FAMILIA_LOGICA')['LT_SEMANAS'].mean().apply(lambda x: f"{x*7:.0f} días"))

✅ Lead Times calculados:
FAMILIA_LOGICA
ACCESORIOS PLASTICOS        5 días
BOTONES Y HERRAJES          5 días
CINTAS Y CORDONES           5 días
CREMALLERAS Y CIERRES       5 días
ELASTICOS                   5 días
ETIQUETAS                   5 días
OTROS INSUMOS               5 días
TEJIDOS (CUELLOS/PUÑOS)    15 días
TEXTILES Y GUATAS           5 días
Name: LT_SEMANAS, dtype: object


## Clasificación de insumos
Clasificamos los insumos cruzando dos dimensiones:

**ABC (Volumen Normalizado):** ¿Cuánto consumen del total en equivalente de prendas? (80% / 15% / 5%)

**XYZ (Recurrencia):** ¿Qué tan frecuentes son? (X ≥60%, Y 30-60%, Z <30% de semanas activas)

### Clasificación ABC (Pareto)

In [218]:
# Ordenar por consumo NORMALIZADO (no por unidades absolutas)
df_calc = df_calc.sort_values('CONSUMO_NORMALIZADO', ascending=False)

# Calcular porcentaje acumulado con datos normalizados
total_gral = df_calc['CONSUMO_NORMALIZADO'].sum()
df_calc['PCT_ACUMULADO'] = df_calc['CONSUMO_NORMALIZADO'].cumsum() / total_gral

# Definir función de clasificación A, B, C
def get_abc(pct):
    if pct <= 0.80: return 'A'
    elif pct <= 0.95: return 'B'
    else: return 'C'

df_calc['CLASE_ABC'] = df_calc['PCT_ACUMULADO'].apply(get_abc)

# Mostrar comparación
print("📊 Clasificación ABC (basada en consumo NORMALIZADO):")
for clase in ['A', 'B', 'C']:
    n = len(df_calc[df_calc['CLASE_ABC'] == clase])
    pct_items = n / len(df_calc) * 100
    print(f"   Clase {clase}: {n} items ({pct_items:.0f}%)")

📊 Clasificación ABC (basada en consumo NORMALIZADO):
   Clase A: 55 items (57%)
   Clase B: 21 items (22%)
   Clase C: 21 items (22%)


### Clasificación XYZ (Estabilidad)

In [219]:
# MEJORA: Umbrales más flexibles para capturar más items en gestión activa
UMBRAL_FREQ_X = 0.60   # 60% o más de las semanas (antes 80%)
UMBRAL_FREQ_Y = 0.30   # Entre 30% y 60% de las semanas (antes 50%)

# X: ≥60% (Frecuente) | Y: 30%-60% (Variable) | Z: <30% (Esporádico)
def get_xyz_freq(pct_semanas):
    if pct_semanas >= UMBRAL_FREQ_X: return 'X'
    elif pct_semanas >= UMBRAL_FREQ_Y: return 'Y'
    else: return 'Z'

df_calc['CLASE_XYZ'] = df_calc['PCT_SEMANAS_CONSUMO'].apply(get_xyz_freq)

# Crear la estrategia combinada (Ej: AX, BY, CZ)
df_calc['ESTRATEGIA'] = df_calc['CLASE_ABC'] + df_calc['CLASE_XYZ']

# Resumen
print("📊 Clasificación XYZ (umbrales flexibles 60%/30%):")
for clase in ['X', 'Y', 'Z']:
    n = len(df_calc[df_calc['CLASE_XYZ'] == clase])
    print(f"   Clase {clase}: {n} items ({n/len(df_calc)*100:.0f}%)")

📊 Clasificación XYZ (umbrales flexibles 60%/30%):
   Clase X: 15 items (15%)
   Clase Y: 13 items (13%)
   Clase Z: 69 items (71%)


## Definición de Reglas (ROP y Stock Seguridad)

In [220]:
from scipy import stats
import warnings

# Ignorar warnings de convergencia en ajustes
warnings.filterwarnings("ignore")

# ═══════════════════════════════════════════════════════════════════════════════
# ANÁLISIS RIGUROSO DE DISTRIBUCIÓN (BEST FIT)
# ═══════════════════════════════════════════════════════════════════════════════

# Lista de distribuciones candidatas a probar
DISTRIBUCIONES_CANDIDATAS = [
    stats.norm,         # Normal (Estándar)
    stats.lognorm,      # LogNormal (Sesgada a derecha, común en demanda)
    stats.gamma,        # Gamma (Muy flexible para tiempos/demanda)
    stats.expon,        # Exponencial (Eventos raros)
    stats.weibull_min   # Weibull (Fallas/Vida útil, a veces demanda)
]

def encontrar_mejor_distribucion(datos):
    """
    Prueba múltiples distribuciones y devuelve la que mejor ajusta (KS Test).
    Retorna: (nombre_distribucion, parametros, p_value)
    """
    # Sumamos un epsilon minúsculo porque lognorm/gamma fallan con ceros exactos
    datos_adj = datos + 1e-6 
    
    mejor_dist = 'DESCONOCIDA'
    mejor_p = -1
    mejores_params = None
    
    # Si la varianza es 0 (consumo constante), no ajustar
    if np.var(datos) == 0:
        return 'CONSTANTE', None, 1.0

    for dist in DISTRIBUCIONES_CANDIDATAS:
        try:
            # 1. Ajustar parámetros (MLE)
            params = dist.fit(datos_adj)
            
            # 2. Test de Bondad de Ajuste (Kolmogorov-Smirnov)
            # H0: Los datos siguen la distribución
            # p-value alto (>0.05) -> No podemos rechazar H0 (Es un buen ajuste)
            D, p_value = stats.kstest(datos_adj, dist.name, args=params)
            
            if p_value > mejor_p:
                mejor_p = p_value
                mejor_dist = dist.name
                mejores_params = params
        except:
            continue
            
    return mejor_dist, mejores_params, mejor_p

def analizar_perfil_demanda_avanzado(row_insumo, datos_semanales):
    """
    Clasifica el perfil usando ajuste estadístico riguroso.
    """
    serie = datos_semanales.loc[row_insumo.name]
    
    # 1. Chequeo de Intermitencia (Regla de Oro para Supply Chain)
    # Si más del 50% son ceros, el ajuste continuo pierde sentido práctico
    pct_ceros = 1 - row_insumo['PCT_SEMANAS_CONSUMO']
    
    if pct_ceros > 0.5:
        return 'LUMPY (INTERMITENTE)', None, 0.0
    
    # 2. Ajuste de Distribución para demanda continua/frecuente
    dist_name, params, p_value = encontrar_mejor_distribucion(serie)
    
    # Si el p-value es muy bajo (<0.01), ninguna distribución teórica ajusta bien
    if p_value < 0.01: 
        return 'ERRÁTICA (SIN AJUSTE)', None, p_value
        
    return dist_name.upper(), params, p_value

# Aplicar análisis (esto puede tardar unos segundos)
print("⏳ Ejecutando pruebas de bondad de ajuste (KS-Test) para cada insumo...")
resultados = df_calc.apply(lambda row: analizar_perfil_demanda_avanzado(row, df_matriz), axis=1, result_type='expand')
df_calc[['PERFIL_DEMANDA', 'DIST_PARAMS', 'P_VALUE']] = resultados

print("📊 Perfiles de Demanda Identificados (Riguroso):")
print(df_calc['PERFIL_DEMANDA'].value_counts())

⏳ Ejecutando pruebas de bondad de ajuste (KS-Test) para cada insumo...
📊 Perfiles de Demanda Identificados (Riguroso):
PERFIL_DEMANDA
LUMPY (INTERMITENTE)     79
ERRÁTICA (SIN AJUSTE)    10
WEIBULL_MIN               4
GAMMA                     3
LOGNORM                   1
Name: count, dtype: int64
📊 Perfiles de Demanda Identificados (Riguroso):
PERFIL_DEMANDA
LUMPY (INTERMITENTE)     79
ERRÁTICA (SIN AJUSTE)    10
WEIBULL_MIN               4
GAMMA                     3
LOGNORM                   1
Name: count, dtype: int64


In [221]:
# ═══════════════════════════════════════════════════════════════════════════════
# Política de gestión basada únicamente en ABC-XYZ
# ═══════════════════════════════════════════════════════════════════════════════

def get_politica_gestion(estrategia):
    """Define política considerando solo estrategia ABC-XYZ"""
    if estrategia in ['AX', 'BX']:
        return 'ROP_AUTOMATICO'
    elif estrategia in ['AY', 'BY', 'CX']:
        return 'REVISION_PERIODICA'
    elif estrategia in ['AZ', 'BZ', 'CY']:
        return 'STOCK_MINIMO'
    else:  # CZ
        return 'BAJO_PEDIDO'

df_calc['POLITICA'] = df_calc['ESTRATEGIA'].apply(get_politica_gestion)

# Nivel de Servicio Objetivo (Probabilidad de NO tener agotados)
NIVEL_SERVICIO = {
    'ROP_AUTOMATICO': 0.95,      # 95% (Críticos)
    'REVISION_PERIODICA': 0.90,  # 90% (Importantes)
    'STOCK_MINIMO': 0.85,        # 85% (Baja rotación)
    'BAJO_PEDIDO': 0.50          # 50% (Sin stock)
}

df_calc['TARGET_SL'] = df_calc['POLITICA'].map(NIVEL_SERVICIO)

# ═══════════════════════════════════════════════════════════════════════════════
# CÁLCULO DE STOCK DE SEGURIDAD (SS) BASADO EN DISTRIBUCIÓN
# ═══════════════════════════════════════════════════════════════════════════════

def calcular_ss_probabilistico(row):
    """
    Calcula SS usando la inversa de la distribución ajustada (PPF)
    para capturar la "cola" de riesgo real (sesgo), en lugar de asumir Normalidad.
    """
    if row['POLITICA'] == 'BAJO_PEDIDO':
        return 0
    
    target_sl = row['TARGET_SL']
    perfil = row['PERFIL_DEMANDA']
    params = row['DIST_PARAMS']
    
    # 1. Calcular el "Pico de Demanda Semanal" según la distribución
    #    (El valor que cubre el X% de las semanas)
    pico_semanal = 0
    
    if perfil == 'LUMPY (INTERMITENTE)' or perfil == 'ERRÁTICA (SIN AJUSTE)':
        # Fallback heurístico para distribuciones no ajustables:
        # Pico = Promedio + 2 Desviaciones (Aprox 95% empírico Chebyshev/Normal)
        # Ajustamos el multiplicador según el nivel de servicio deseado
        z_factor = 2.0 if target_sl >= 0.95 else 1.65
        pico_semanal = row['PROMEDIO_SEM'] + (z_factor * row['DESV_STD'])
        
    elif perfil == 'NORM':
        # Normal: loc=mean, scale=std
        pico_semanal = stats.norm.ppf(target_sl, *params)
        
    elif perfil == 'LOGNORM':
        # Lognormal: s, loc, scale
        pico_semanal = stats.lognorm.ppf(target_sl, *params)
        
    elif perfil == 'GAMMA':
        # Gamma: a, loc, scale
        pico_semanal = stats.gamma.ppf(target_sl, *params)
        
    elif perfil == 'EXPON':
        # Exponencial: loc, scale
        pico_semanal = stats.expon.ppf(target_sl, *params)
        
    elif perfil == 'WEIBULL_MIN':
        # Weibull: c, loc, scale
        pico_semanal = stats.weibull_min.ppf(target_sl, *params)
        
    else:
        # Default
        pico_semanal = row['PROMEDIO_SEM'] + (1.64 * row['DESV_STD'])

    # 2. Calcular Stock de Seguridad
    #    SS = (Pico Semanal - Promedio Semanal) * Raíz(Lead Time)
    #    Esta fórmula escala la variabilidad semanal al horizonte del Lead Time
    variabilidad_semanal = max(0, pico_semanal - row['PROMEDIO_SEM'])
    ss = variabilidad_semanal * np.sqrt(row['LT_SEMANAS'])
    
    return ss

# 1. Calcular Stock de Seguridad Probabilístico
df_calc['STOCK_SEGURIDAD'] = df_calc.apply(calcular_ss_probabilistico, axis=1)

# 2. Punto de Reorden (ROP) = Demanda Promedio en LT + Stock Seguridad
df_calc['PUNTO_REORDEN'] = (df_calc['PROMEDIO_SEM'] * df_calc['LT_SEMANAS']) + df_calc['STOCK_SEGURIDAD']

# Redondear
df_calc[['STOCK_SEGURIDAD', 'PUNTO_REORDEN']] = df_calc[['STOCK_SEGURIDAD', 'PUNTO_REORDEN']].round(0)

# Resumen
print("📋 DISTRIBUCIÓN DE POLÍTICAS Y ROP:")
print("="*60)
print(df_calc.groupby(['POLITICA', 'PERFIL_DEMANDA'])['PUNTO_REORDEN'].count())

📋 DISTRIBUCIÓN DE POLÍTICAS Y ROP:
POLITICA            PERFIL_DEMANDA       
BAJO_PEDIDO         LUMPY (INTERMITENTE)     21
REVISION_PERIODICA  ERRÁTICA (SIN AJUSTE)     3
                    LUMPY (INTERMITENTE)     10
ROP_AUTOMATICO      ERRÁTICA (SIN AJUSTE)     7
                    GAMMA                     3
                    LOGNORM                   1
                    WEIBULL_MIN               4
STOCK_MINIMO        LUMPY (INTERMITENTE)     48
Name: PUNTO_REORDEN, dtype: int64


## Visualizaciones de Resultados

A continuación se presentan gráficos diseñados para explicar y comunicar los resultados del análisis de reglas de reabastecimiento a diferentes audiencias.

### Matriz ABC-XYZ: Mapa Estratégico de Insumos
Esta matriz cruza **volumen de consumo (ABC)** con **frecuencia de uso (XYZ)** para definir estrategias de reabastecimiento diferenciadas.

In [222]:
# GRÁFICO 1: Matriz ABC-XYZ con Políticas de Gestión (Heatmap interactivo) - CORREGIDO
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy as np

# --- (Asumiendo que df_calc ya está calculado en tu entorno previo) ---
# Si necesitas probarlo aislado, descomenta estas líneas para generar datos dummy:
# df_calc = pd.DataFrame({
#     'CLASE_ABC': ['A']*28 + ['B']*10 + ['C']*5,
#     'CLASE_XYZ': ['X']*15 + ['Y']*13 + ['Z']*15,
#     'POLITICA': ['ROP_AUTOMATICO']*43
# })

# Crear tabla cruzada de estrategias
matriz_estrategia = pd.crosstab(df_calc['CLASE_ABC'], df_calc['CLASE_XYZ'])
matriz_estrategia = matriz_estrategia.reindex(index=['A','B','C'], columns=['X','Y','Z'], fill_value=0)

# Crear matriz de prioridades para el color (AX=0, CZ=1)
prioridad = np.array([[0, 0.2, 0.4], [0.3, 0.5, 0.7], [0.6, 0.8, 1.0]])

# Mapeo de políticas para cada celda
politica_map = {
    'AX': 'ROP AUTOMÁTICO', 'BX': 'ROP AUTOMÁTICO',
    'AY': 'REVISIÓN PERIÓDICA', 'BY': 'REVISIÓN PERIÓDICA',
    'AZ': 'STOCK MÍNIMO', 'BZ': 'STOCK MÍNIMO',
    'CX': 'STOCK MÍNIMO', 'CY': 'BAJO PEDIDO', 'CZ': 'BAJO PEDIDO'
}

# Crear texto enriquecido para cada celda
text_matrix = []
for i, abc in enumerate(['A', 'B', 'C']):
    row = []
    for j, xyz in enumerate(['X', 'Y', 'Z']):
        valor = matriz_estrategia.loc[abc, xyz]
        estrategia = f"{abc}{xyz}"
        politica = politica_map[estrategia]
        # CORRECCIÓN: Usamos <br> extra para dar aire y reducimos el tamaño en el layout
        row.append(f"<b>{estrategia}</b><br>{valor} items<br><span style='font-size:10px'>{politica}</span>")
    text_matrix.append(row)

fig1 = go.Figure(data=go.Heatmap(
    z=prioridad,
    x=['X<br>(Frecuente ≥80%)', 'Y<br>(Variable 50-80%)', 'Z<br>(Esporádico <50%)'],
    y=['A<br>(80% vol)', 'B<br>(15% vol)', 'C<br>(5% vol)'],
    text=text_matrix,
    texttemplate="%{text}",
    textfont={"size": 11}, # CORRECCIÓN: Reducido de 13 a 11 para evitar solapamiento interno
    colorscale=[[0, '#2ecc71'], [0.3, '#91cf60'], [0.5, '#fee08b'], [0.7, '#fdae61'], [1, '#e74c3c']],
    showscale=False,
    hovertemplate='Estrategia: %{text}<extra></extra>'
))

fig1.update_layout(
    title=dict(text='Matriz ABC-XYZ: Estrategia de Reabastecimiento', font=dict(size=18)),
    xaxis_title='Frecuencia de Consumo (XYZ)',
    yaxis_title='Volumen Normalizado (ABC)',
    height=550, # Aumentado ligeramente para dar espacio vertical
    template='plotly_white',
    margin=dict(r=260, t=80, b=80), # CORRECCIÓN: Margen derecho generoso para la leyenda
    annotations=[
        dict(
            text=(
                "<b>POLÍTICAS DE GESTIÓN:</b><br><br>" +
                "🟢 <b>ROP AUTOMÁTICO</b><br>   (AX, BX)<br><br>" +
                "🟡 <b>REVISIÓN PERIÓDICA</b><br>   (AY, BY)<br><br>" +
                "🟠 <b>STOCK MÍNIMO</b><br>   (AZ, BZ, CX)<br><br>" +
                "🔴 <b>BAJO PEDIDO</b><br>   (CY, CZ)<br><br>" +
                "<i>*ABC usa consumo<br>normalizado</i>"
            ),
            xref="paper", yref="paper",
            x=1.02,  # CORRECCIÓN: Movido de 1.02 a 1.15 para sacarlo del gráfico
            y=0.5,
            showarrow=False,
            xanchor="left",
            yanchor="middle",
            font=dict(size=11),
            align="left",
            bgcolor="#f8f9fa",
            bordercolor="#dee2e6",
            borderwidth=1,
            width=170 # Definir ancho fijo ayuda a controlar el layout
        )
    ]
)

fig1.show()

### 2️⃣ Curva de Pareto ABC: Concentración del Consumo
Visualiza cómo un pequeño porcentaje de insumos representa la mayor parte del consumo total (Principio 80/20).

In [223]:
df_calc.sort_values('CONSUMO_NORMALIZADO', ascending=False).reset_index()

,INSUMO,CONSUMO_TOTAL,PROMEDIO_SEM,DESV_STD,CV,PCT_SEMANAS_CONSUMO,FAMILIA_LOGICA,LOG_CONSUMO,MAX_LOG_FAM,MIN_LOG_FAM,...,CLASE_ABC,CLASE_XYZ,ESTRATEGIA,PERFIL_DEMANDA,DIST_PARAMS,P_VALUE,POLITICA,TARGET_SL,STOCK_SEGURIDAD,PUNTO_REORDEN
0,BOE0028 BOTON ESTANDAR,264374.00,9791.629630,11904.651771,1.215799,1.000000,BOTONES Y HERRAJES,12.485124,12.485124,3.218876,...,A,X,AX,LOGNORM,"(1.006997314112826, -314.1570348224167, 6076.4...",0.919130,ROP_AUTOMATICO,0.95,18370.0,25364.0
1,CIR0024-CINTA REFLECTIVA 2 PULG,20018.89,741.440370,1257.342709,1.695811,0.925926,CINTAS Y CORDONES,9.904482,9.904482,1.472472,...,A,X,AX,GAMMA,"(0.39935027972496895, 9.999999999999997e-07, 7...",0.282355,ROP_AUTOMATICO,0.95,487.0,1016.0
2,TMA0104-MALE,22.00,0.814815,4.233902,5.196152,0.037037,OTROS INSUMOS,3.135494,3.135494,2.763170,...,A,Z,AZ,LUMPY (INTERMITENTE),None,0.000000,STOCK_MINIMO,0.85,6.0,6.0
3,TAP0034 TANCA PUNTERA,555.00,20.555556,51.995809,2.529526,0.370370,ACCESORIOS PLASTICOS,6.320768,6.320768,3.258097,...,A,Y,AY,LUMPY (INTERMITENTE),None,0.000000,REVISION_PERIODICA,0.90,73.0,87.0
4,REC0017-RESORTE CAUCHO 3 CM,8388.31,310.678148,450.866181,1.451232,0.925926,ELASTICOS,9.034714,9.034714,3.055886,...,A,X,AX,GAMMA,"(0.5267255128148094, 9.999999999999997e-07, 45...",0.829224,ROP_AUTOMATICO,0.95,503.0,725.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,HILADILLLO,14.85,0.550000,2.857884,5.196152,0.037037,OTROS INSUMOS,2.763170,3.135494,2.763170,...,C,Z,CZ,LUMPY (INTERMITENTE),None,0.000000,BAJO_PEDIDO,0.50,0.0,0.0
93,GUA0027-GUATA ACOLCHADA 300,12.20,0.451852,2.347891,5.196152,0.037037,TEXTILES Y GUATAS,2.580217,7.450196,2.580217,...,C,Z,CZ,LUMPY (INTERMITENTE),None,0.000000,BAJO_PEDIDO,0.50,0.0,0.0
94,MAP098 MARQUILLA PERSONALIZADA,140.00,5.185185,26.943013,5.196152,0.037037,ETIQUETAS,4.948760,9.541656,4.948760,...,C,Z,CZ,LUMPY (INTERMITENTE),None,0.000000,BAJO_PEDIDO,0.50,0.0,4.0
95,HIC0020-HILO CAUCHO DELGADO,20.24,0.749630,3.847358,5.132345,0.111111,ELASTICOS,3.055886,9.034714,3.055886,...,C,Z,CZ,LUMPY (INTERMITENTE),None,0.000000,BAJO_PEDIDO,0.50,0.0,1.0


In [224]:
# GRÁFICO 2: Curva de Pareto ABC interactiva (SCORE NORMALIZADO) - CORREGIDO
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd

# --- (Bloque de datos simulados para que el código sea ejecutable independientemente) ---
# Si ya tienes df_calc, puedes ignorar o comentar este bloque if.
if 'df_calc' not in locals():
    # Creamos un df_calc dummy para demostración
    df_calc = pd.DataFrame({
        'INSUMO': [f'Item {i}' for i in range(1, 101)],
        'CONSUMO_NORMALIZADO': [1000 - i*10 for i in range(100)], # Decreciente
        'CONSUMO_TOTAL': [1000 - i*10 for i in range(100)],
        'CLASE_ABC': ['A']*20 + ['B']*30 + ['C']*50,
        'FAMILIA_LOGICA': ['General']*100
    })

# -------------------------------------------------------------------------------------

# 1. Preparar datos
df_pareto = df_calc.sort_values('CONSUMO_NORMALIZADO', ascending=False).reset_index()
df_pareto['Item_Num'] = range(1, len(df_pareto) + 1)
# Calcular % acumulado
df_pareto['PCT_ACUM_CONSUMO'] = df_pareto['CONSUMO_NORMALIZADO'].cumsum() / df_pareto['CONSUMO_NORMALIZADO'].sum() * 100

# Colores por clase ABC
color_map_abc = {'A': '#2ecc71', 'B': '#f39c12', 'C': '#e74c3c'}
df_pareto['Color'] = df_pareto['CLASE_ABC'].map(color_map_abc)

# Crear figura con doble eje Y
fig2 = make_subplots(specs=[[{"secondary_y": True}]])

# 2. Barras de Score Normalizado (Eje Primario)
fig2.add_trace(
    go.Bar(
        x=df_pareto['Item_Num'], 
        y=df_pareto['CONSUMO_NORMALIZADO'],
        marker_color=df_pareto['Color'], 
        name='Score Importancia',
        # Customdata para el hover (agregamos protección si faltan columnas)
        customdata=df_pareto[['INSUMO', 'CLASE_ABC', 'CONSUMO_TOTAL', 'FAMILIA_LOGICA']].values,
        hovertemplate=(
            '<b>%{customdata[0]}</b><br>' +
            'Familia: %{customdata[3]}<br>' +
            'Score: %{y:.1f} pts<br>' +
            'Consumo Real: %{customdata[2]:,.0f}<br>' +
            'Clase: %{customdata[1]}<extra></extra>'
        )
    ),
    secondary_y=False
)

# 3. Línea de % acumulado (Eje Secundario)
fig2.add_trace(
    go.Scatter(
        x=df_pareto['Item_Num'], 
        y=df_pareto['PCT_ACUM_CONSUMO'],
        mode='lines', 
        name='% Acumulado', 
        line=dict(color='#3498db', width=3),
        hovertemplate='Item #%{x}<br>% Acumulado: %{y:.1f}%<extra></extra>'
    ),
    secondary_y=True
)

# 4. Líneas de corte (Sin texto automático)
fig2.add_hline(y=80, line_dash="dash", line_color="#27ae60", secondary_y=True)
fig2.add_hline(y=95, line_dash="dash", line_color="#f39c12", secondary_y=True)

# --- ELIMINAMOS LAS ANOTACIONES EXTERNAS E INCORPORAMOS A LA LEYENDA ---

# 5. Coloreado de Zonas (Fondo)
# Encontramos los índices de corte
try:
    idx_80 = df_pareto[df_pareto['PCT_ACUM_CONSUMO'] >= 80].index[0] + 1
    idx_95 = df_pareto[df_pareto['PCT_ACUM_CONSUMO'] >= 95].index[0] + 1
except IndexError:
    idx_80 = len(df_pareto) # Fallback si no llega al 80%
    idx_95 = len(df_pareto)

fig2.add_vrect(x0=0, x1=idx_80, fillcolor="#2ecc71", opacity=0.1, line_width=0)
fig2.add_vrect(x0=idx_80, x1=idx_95, fillcolor="#f39c12", opacity=0.1, line_width=0)
fig2.add_vrect(x0=idx_95, x1=len(df_pareto)+1, fillcolor="#e74c3c", opacity=0.1, line_width=0)

# --- ETIQUETAS DE ZONA (A, B, C) ---
y_max_barras = df_pareto['CONSUMO_NORMALIZADO'].max()

# Etiqueta Clase A
fig2.add_annotation(x=idx_80/2, y=y_max_barras, text="<b>A</b>", showarrow=False, font=dict(color="#27ae60", size=15), yanchor="top")

# Etiqueta Clase B (solo si existe zona B)
if idx_95 > idx_80:
    fig2.add_annotation(x=idx_80 + (idx_95-idx_80)/2, y=y_max_barras, text="<b>B</b>", showarrow=False, font=dict(color="#f39c12", size=15), yanchor="top")

# Etiqueta Clase C (solo si existe zona C)
if len(df_pareto) > idx_95:
    fig2.add_annotation(x=idx_95 + (len(df_pareto)-idx_95)/2, y=y_max_barras, text="<b>C</b>", showarrow=False, font=dict(color="#e74c3c", size=15), yanchor="top")


# 6. Layout Final
fig2.update_layout(
    title=dict(text='Análisis de Pareto: Clasificación ABC', font=dict(size=18)),
    xaxis_title='Insumos (ordenados por Score de Importancia)',
    height=550,
    template='plotly_white',
    # Leyenda: Mostramos las líneas de límite como trazas fantasma para que aparezcan en la leyenda
    showlegend=True,
    legend=dict(
        orientation="h", 
        yanchor="bottom", y=1.02, 
        xanchor="right", x=1 # Llevamos la leyenda al borde derecho
    ),
    hovermode='x unified',
    # Eliminamos el margen superior extra y el derecho ajustado manualmente
    margin=dict(r=50, t=50) 
)

# Incorporar las líneas de límite al legend (Trazas fantasma)
fig2.add_trace(
    go.Scatter(
        x=[None], y=[None],
        mode='lines',
        line=dict(color="#27ae60", dash="dash"),
        name='Límite 80% (Clase A)'
    ),
    secondary_y=True
)
fig2.add_trace(
    go.Scatter(
        x=[None], y=[None],
        mode='lines',
        line=dict(color="#f39c12", dash="dash"),
        name='Límite 95% (Clase B)'
    ),
    secondary_y=True
)


fig2.update_yaxes(title_text="Score de Importancia (Normalizado)", secondary_y=False)
fig2.update_yaxes(title_text="% Acumulado", secondary_y=True, range=[0, 105])

fig2.show()

# Estadísticas
n_A = len(df_calc[df_calc['CLASE_ABC']=='A'])
n_B = len(df_calc[df_calc['CLASE_ABC']=='B'])
n_C = len(df_calc[df_calc['CLASE_ABC']=='C'])
pct_vol_A = df_calc[df_calc['CLASE_ABC']=='A']['CONSUMO_NORMALIZADO'].sum() / df_calc['CONSUMO_NORMALIZADO'].sum() * 100
print(f"\n📊 Distribución ABC normalizada: A={n_A} items ({n_A/len(df_calc)*100:.0f}%) | B={n_B} items ({n_B/len(df_calc)*100:.0f}%) | C={n_C} items ({n_C/len(df_calc)*100:.0f}%)")
print(f"   → Los {n_A} items Clase A representan {pct_vol_A:.1f}% del Score de Importancia")


📊 Distribución ABC normalizada: A=55 items (57%) | B=21 items (22%) | C=21 items (22%)
   → Los 55 items Clase A representan 79.8% del Score de Importancia


### 3️⃣ Scatter Plot: Volumen vs Frecuencia
Cada punto es un insumo. Permite identificar visualmente en qué cuadrante estratégico cae cada uno.

In [225]:
# GRÁFICO 3: Scatter Plot interactivo - Volumen vs Frecuencia con Políticas
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy as np

# --- (Bloque de datos simulados para ejecución independiente) ---
if 'df_calc' not in locals():
    np.random.seed(42)
    N = 100
    df_calc = pd.DataFrame({
        'INSUMO': [f'Item {i}' for i in range(N)],
        'CONSUMO_NORMALIZADO': np.random.lognormal(mean=7, sigma=1.5, size=N),
        'CONSUMO_TOTAL': np.random.randint(100, 5000, size=N),
        'PCT_SEMANAS_CONSUMO': np.random.rand(N), # 0 a 1
        'FAMILIA_LOGICA': np.random.choice(['Familia A', 'Familia B', 'Familia C'], size=N)
    })
    
    # Asignar política basada en umbrales del ejemplo (60% y 30%)
    def assign_politica(row):
        freq = row['PCT_SEMANAS_CONSUMO'] * 100
        if freq >= 60: return 'ROP_AUTOMATICO'
        elif freq >= 30: return 'REVISION_PERIODICA'
        else: return 'BAJO_PEDIDO'
    
    df_calc['POLITICA'] = df_calc.apply(assign_politica, axis=1)
    # Algunos casos especiales para stock mínimo
    df_calc.loc[df_calc['CONSUMO_NORMALIZADO'] > 8000, 'POLITICA'] = 'STOCK_MINIMO'

# -------------------------------------------------------------------------------------

# Preparar datos
df_scatter = df_calc.reset_index().copy()
df_scatter['PCT_FREQ_100'] = df_scatter['PCT_SEMANAS_CONSUMO'] * 100
df_scatter['INSUMO_SHORT'] = df_scatter['INSUMO'].apply(lambda x: x[:30] + '...' if len(x) > 30 else x)

# Umbrales (Según tu código ejemplo: 60% y 30%)
UMBRAL_X_Y = 60
UMBRAL_Y_Z = 30

# Colores por política
color_politica = {
    'ROP_AUTOMATICO': '#2ecc71',      # Verde
    'REVISION_PERIODICA': '#f1c40f',  # Amarillo
    'STOCK_MINIMO': '#e67e22',        # Naranja
    'BAJO_PEDIDO': '#95a5a6'          # Gris
}

fig3 = px.scatter(
    df_scatter,
    x='PCT_FREQ_100',
    y='CONSUMO_NORMALIZADO',
    color='POLITICA',
    size='CONSUMO_NORMALIZADO',
    hover_name='INSUMO_SHORT',
    hover_data={
        'CONSUMO_NORMALIZADO': ':.1f',
        'CONSUMO_TOTAL': ':,.0f',
        'FAMILIA_LOGICA': True,
        'PCT_FREQ_100': ':.1f',
        'POLITICA': True,
        'INSUMO_SHORT': False
    },
    color_discrete_map=color_politica,
    category_orders={'POLITICA': ['ROP_AUTOMATICO','REVISION_PERIODICA','STOCK_MINIMO','BAJO_PEDIDO']},
    labels={
        'PCT_FREQ_100': 'Frecuencia (% semanas activas)',
        'CONSUMO_NORMALIZADO': 'Score de Importancia (Volumen)',
        'CONSUMO_TOTAL': 'Consumo Real',
        'FAMILIA_LOGICA': 'Familia'
    }
)

# Escala logarítmica si es necesario
if df_scatter['CONSUMO_NORMALIZADO'].max() / df_scatter['CONSUMO_NORMALIZADO'].min() > 100:
    fig3.update_yaxes(type="log", title="Score de Importancia (escala log)")

# --- 1. ZONAS SOMBREADAS DE FONDO (Igual que Pareto) ---
# Usamos vrect para colorear todo el fondo verticalmente
# Zona Z (Baja Frecuencia < 30%) - Rojo tenue
fig3.add_vrect(x0=-5, x1=UMBRAL_Y_Z, fillcolor="#e74c3c", opacity=0.1, line_width=0, layer="below")
# Zona Y (Media Frecuencia 30-60%) - Amarillo tenue
fig3.add_vrect(x0=UMBRAL_Y_Z, x1=UMBRAL_X_Y, fillcolor="#f39c12", opacity=0.1, line_width=0, layer="below")
# Zona X (Alta Frecuencia > 60%) - Verde tenue
fig3.add_vrect(x0=UMBRAL_X_Y, x1=105, fillcolor="#2ecc71", opacity=0.1, line_width=0, layer="below")

# --- 2. LÍNEAS DE UMBRAL Y ETIQUETAS DE % ---
# Línea 60%
fig3.add_vline(x=UMBRAL_X_Y, line_dash="dash", line_color="#27ae60", line_width=2)
fig3.add_annotation(
    x=UMBRAL_X_Y, y=1, yref="paper", 
    text=f"<b>{UMBRAL_X_Y}%</b>", 
    showarrow=False, 
    font=dict(color="#27ae60", size=11), 
    yshift=10, 
    bgcolor="rgba(255,255,255,0.8)"
)

# Línea 30%
fig3.add_vline(x=UMBRAL_Y_Z, line_dash="dash", line_color="#e74c3c", line_width=2)
fig3.add_annotation(
    x=UMBRAL_Y_Z, y=1, yref="paper", 
    text=f"<b>{UMBRAL_Y_Z}%</b>", 
    showarrow=False, 
    font=dict(color="#e74c3c", size=11), 
    yshift=10, 
    bgcolor="rgba(255,255,255,0.8)"
)

# --- 3. ETIQUETAS GRANDES DE ZONA (X, Y, Z) ---
# Usamos yref="paper" para que siempre estén arriba sin importar la escala log/lineal
y_zona_label = 0.99 # Posición vertical (99% de la altura del gráfico)

# Etiqueta Z (Izquierda)
fig3.add_annotation(
    x=UMBRAL_Y_Z / 2, y=y_zona_label, yref="paper",
    text="<b>Z</b>", showarrow=False, 
    font=dict(size=24, color="rgba(231, 76, 60, 0.4)") # Color semitransparente para no tapar puntos
)

# Etiqueta Y (Centro)
fig3.add_annotation(
    x=(UMBRAL_Y_Z + UMBRAL_X_Y) / 2, y=y_zona_label, yref="paper",
    text="<b>Y</b>", showarrow=False, 
    font=dict(size=24, color="rgba(243, 156, 18, 0.4)")
)

# Etiqueta X (Derecha)
fig3.add_annotation(
    x=(UMBRAL_X_Y + 100) / 2, y=y_zona_label, yref="paper",
    text="<b>X</b>", showarrow=False, 
    font=dict(size=24, color="rgba(46, 204, 113, 0.4)")
)

# --- 4. LAYOUT Y CORRECCIÓN DE LEYENDA ---
fig3.update_layout(
    title=dict(text='Mapa de Insumos: Políticas de Gestión', font=dict(size=18)),
    template='plotly_white',
    height=600,
    xaxis=dict(range=[-2, 102], title='Frecuencia (% semanas activas)'),
    
    # LEYENDA: Movida arriba a la derecha, fuera del área de trazado pero alineada
    legend=dict(
        title=None, 
        orientation='h', 
        yanchor='bottom', 
        y=1.02,          # Justo encima del gráfico
        xanchor='right', 
        x=1,             # Alineada a la derecha
        bgcolor='rgba(255,255,255,0.5)'
    ),
    # Aumentamos margen superior para que quepan título, leyenda y etiquetas de %
    margin=dict(t=80, r=30)
)

fig3.show()

### 4️⃣ Top 15 Insumos Críticos con Punto de Reorden
Muestra los insumos más importantes que requieren gestión activa de inventario con sus parámetros de control.

In [226]:
# GRÁFICO 4: Top 15 Insumos con ROP (Barras horizontales apiladas)
df_rop = df_calc[df_calc['PUNTO_REORDEN'] > 0].nlargest(15, 'PUNTO_REORDEN').copy()

if len(df_rop) > 0:
    df_rop = df_rop.reset_index()
    df_rop['INSUMO_SHORT'] = df_rop['INSUMO'].apply(lambda x: x[:35] + '...' if len(x) > 35 else x)
    df_rop['DEMANDA_LT'] = df_rop['PROMEDIO_SEM'] * df_rop['LT_SEMANAS']
    
    # Ordenar de mayor a menor ROP
    df_rop = df_rop.sort_values('PUNTO_REORDEN', ascending=True)
    
    fig4 = go.Figure()
    
    # Barras: Demanda durante Lead Time
    fig4.add_trace(go.Bar(
        y=df_rop['INSUMO_SHORT'],
        x=df_rop['DEMANDA_LT'],
        name='Demanda durante Lead Time',
        orientation='h',
        marker_color='#3498db',
        hovertemplate='<b>%{y}</b><br>Demanda LT: %{x:,.0f}<extra></extra>'
    ))
    
    # Barras: Stock de Seguridad
    fig4.add_trace(go.Bar(
        y=df_rop['INSUMO_SHORT'],
        x=df_rop['STOCK_SEGURIDAD'],
        name='Stock de Seguridad',
        orientation='h',
        marker_color='#e74c3c',
        hovertemplate='<b>%{y}</b><br>Stock Seg: %{x:,.0f}<extra></extra>'
    ))
    
    # Anotaciones con ROP y estrategia
    for i, row in df_rop.iterrows():
        rop = row['PUNTO_REORDEN']
        estrategia = row['ESTRATEGIA']
        fig4.add_annotation(
            x=rop + df_rop['PUNTO_REORDEN'].max() * 0.02,
            y=row['INSUMO_SHORT'],
            text=f"<b>ROP: {int(rop):,}</b> [{estrategia}]",
            showarrow=False,
            font=dict(size=10),
            xanchor='left'
        )
    
    fig4.update_layout(
        barmode='stack',
        title=dict(text='📦 Top 15 Insumos: Componentes del Punto de Reorden (ROP)<br><sup>ROP = Demanda durante Lead Time + Stock de Seguridad</sup>', 
                   font=dict(size=18)),
        xaxis_title='Cantidad (unidades)',
        yaxis_title='',
        template='plotly_white',
        height=550,
        legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='center', x=0.5),
        xaxis=dict(range=[0, df_rop['PUNTO_REORDEN'].max() * 1.25])
    )
    
    fig4.show()
else:
    print("No hay insumos con Punto de Reorden definido.")

### 5️⃣ Dashboard Resumen Ejecutivo
Vista consolidada con métricas clave y distribución de estrategias para presentación a directivos.

In [237]:
# GRÁFICO 5: Dashboard Resumen Ejecutivo con Plotly (Actualizado con Políticas)
from plotly.subplots import make_subplots

# Calcular métricas
total_insumos = len(df_calc)
pct_vol_A = df_calc[df_calc['CLASE_ABC']=='A']['CONSUMO_NORMALIZADO'].sum() / df_calc['CONSUMO_NORMALIZADO'].sum() * 100

# Conteo por política
n_rop = len(df_calc[df_calc['POLITICA'] == 'ROP_AUTOMATICO'])
n_rev = len(df_calc[df_calc['POLITICA'] == 'REVISION_PERIODICA'])
n_stock = len(df_calc[df_calc['POLITICA'] == 'STOCK_MINIMO'])
n_pedido = len(df_calc[df_calc['POLITICA'] == 'BAJO_PEDIDO'])

# Crear subplots
fig5 = make_subplots(
    rows=2, cols=3,
    specs=[[{"type": "indicator"}, {"type": "pie"}, {"type": "pie"}],
           [{"type": "bar", "colspan": 2}, None, {"type": "table"}]],
    subplot_titles=('', 'Distribución ABC (Vol. Normalizado)', 'Distribución XYZ (Frecuencia)', 
                    'Items por Estrategia ABC-XYZ', ''),
    vertical_spacing=0.12,
    horizontal_spacing=0.08
)

# ═══════════════════════════════════════════════════════════
# PANEL 1: KPI de Insumos con Gestión Activa
# ═══════════════════════════════════════════════════════════
insumos_gestion_activa = n_rop + n_rev + n_stock
fig5.add_trace(go.Indicator(
    mode="number+delta",
    value=insumos_gestion_activa,
    delta={'reference': total_insumos, 'relative': True, 'valueformat': '.0%'},
    title={"text": "📦 Gestión Activa<br><span style='font-size:0.7em'>de " + str(total_insumos) + " insumos</span>"},
    number={'font': {'size': 50, 'color': '#27ae60'}}
), row=1, col=1)

# ═══════════════════════════════════════════════════════════
# PANEL 2: Donut ABC
# ═══════════════════════════════════════════════════════════
abc_counts = df_calc['CLASE_ABC'].value_counts().reindex(['A','B','C']).fillna(0)
fig5.add_trace(go.Pie(
    labels=['A (80%)', 'B (15%)', 'C (5%)'],
    values=abc_counts.values,
    hole=0.5,
    marker_colors=['#2ecc71', '#f39c12', '#e74c3c'],
    textinfo='label+percent',
    textfont_size=11,
    hovertemplate='Clase %{label}<br>%{value} items<br>%{percent}<extra></extra>'
), row=1, col=2)

# ═══════════════════════════════════════════════════════════
# PANEL 3: Donut XYZ
# ═══════════════════════════════════════════════════════════
xyz_counts = df_calc['CLASE_XYZ'].value_counts().reindex(['X','Y','Z']).fillna(0)
fig5.add_trace(go.Pie(
    labels=['X (≥60%)', 'Y (30-60%)', 'Z (<30%)'],
    values=xyz_counts.values,
    hole=0.5,
    marker_colors=['#27ae60', '#f1c40f', '#e74c3c'],
    textinfo='label+percent',
    textfont_size=11,
    hovertemplate='Clase %{label}<br>%{value} items<br>%{percent}<extra></extra>'
), row=1, col=3)

# ═══════════════════════════════════════════════════════════
# PANEL 4: Barras por Estrategia
# ═══════════════════════════════════════════════════════════
estrategias_order = ['AX','AY','AZ','BX','BY','BZ','CX','CY','CZ']
estrategia_counts = df_calc['ESTRATEGIA'].value_counts().reindex(estrategias_order).fillna(0)
colors_estrategia = ['#1a9850', '#91cf60', '#d9ef8b', '#66bd63', '#fee08b', '#fdae61', '#a6d96a', '#fc8d59', '#d73027']

fig5.add_trace(go.Bar(
    x=estrategias_order,
    y=estrategia_counts.values,
    marker_color=colors_estrategia,
    text=estrategia_counts.values.astype(int),
    textposition='inside',
    hovertemplate='Estrategia %{x}<br>%{y} insumos<extra></extra>'
), row=2, col=1)

# ═══════════════════════════════════════════════════════════
# PANEL 5: Tabla de Políticas de Gestión
# ═══════════════════════════════════════════════════════════
fig5.add_trace(go.Table(
    header=dict(values=['<b>Política</b>', '<b>Items</b>', '<b>Estrategias</b>'],
                fill_color='#3498db', font=dict(color='white', size=11),
                align='left'),
    cells=dict(values=[
        ['🟢 ROP Automático', '🟡 Revisión Periódica', '🟠 Stock Mínimo', '⚪ Bajo Pedido'],
        [n_rop, n_rev, n_stock, n_pedido],
        ['AX, BX', 'AY, BY, CX', 'AZ, BZ, CY', 'CZ']
    ],
    fill_color=[['#e8f8f5', '#fef9e7', '#fdebd0', '#f5f5f5']*3],
    font=dict(size=10),
    align='left',
    height=25)
), row=2, col=3)

fig5.update_layout(
    title=dict(text='DASHBOARD: Reglas de Reabastecimiento de Insumos (Metodología Normalizada)', 
               font=dict(size=20), x=0.5),
    template='plotly_white',
    height=700,
    showlegend=False,
    annotations=[
        dict(text=f"<b>ABC usa consumo normalizado (equiv. prendas)</b> | Clase A representa {pct_vol_A:.0f}% del volumen",
             xref="paper", yref="paper", x=0.5, y=-0.1, showarrow=False, font=dict(size=11))
    ]
)

fig5.show()

### 6️⃣ Heatmap Temporal: Patrón de Consumo por Semana
Visualiza la estacionalidad y patrones de consumo de los insumos más importantes a lo largo del tiempo.

In [228]:
# GRÁFICO 6: Heatmap Temporal interactivo - Patrón de Consumo
# Seleccionar top 20 insumos clase A y B
top_insumos = df_calc[df_calc['CLASE_ABC'].isin(['A','B'])].nlargest(20, 'CONSUMO_TOTAL').index.tolist()

if len(top_insumos) > 0:
    # Preparar matriz normalizada
    df_heatmap = df_matriz.loc[top_insumos].copy()
    df_heatmap_norm = df_heatmap.div(df_heatmap.max(axis=1), axis=0).fillna(0)
    
    # Nombres cortos para visualización
    nombres_cortos = [n[:30] + '...' if len(n) > 30 else n for n in top_insumos]
    
    # Crear hovertext con info adicional
    hovertext = []
    for i, insumo in enumerate(top_insumos):
        row_text = []
        for j, semana in enumerate(df_heatmap.columns):
            consumo = df_heatmap.iloc[i, j]
            estrategia = df_calc.loc[insumo, 'ESTRATEGIA']
            row_text.append(f"<b>{insumo[:25]}</b><br>Semana: {semana}<br>Consumo: {consumo:,.0f}<br>Estrategia: {estrategia}")
        hovertext.append(row_text)
    
    fig6 = go.Figure(data=go.Heatmap(
        z=df_heatmap_norm.values,
        x=[f'S{s}' for s in df_heatmap.columns],
        y=nombres_cortos,
        colorscale='Blues',
        hovertext=hovertext,
        hovertemplate='%{hovertext}<extra></extra>',
        colorbar=dict(title='Intensidad<br>Relativa', tickformat='.0%')
    ))
    
    # Añadir estrategias como anotaciones
    estrategias_text = [df_calc.loc[insumo, 'ESTRATEGIA'] for insumo in top_insumos]
    
    fig6.update_layout(
        title=dict(text='🗓️ Patrón de Consumo Semanal - Top 20 Insumos (Clase A y B)<br><sup>Intensidad normalizada: blanco=0, azul oscuro=máximo consumo</sup>',
                   font=dict(size=18)),
        xaxis_title='Semana del Año',
        yaxis_title='Insumo',
        template='plotly_white',
        height=650,
        yaxis=dict(tickmode='array', tickvals=list(range(len(nombres_cortos))), ticktext=nombres_cortos),
        xaxis=dict(tickangle=45)
    )
    
    # Añadir etiquetas de estrategia al lado derecho
    for i, (nombre, estrategia) in enumerate(zip(nombres_cortos, estrategias_text)):
        color = {'AX': '#1a9850', 'AY': '#91cf60', 'AZ': '#d9ef8b', 
                 'BX': '#66bd63', 'BY': '#fee08b', 'BZ': '#fdae61'}.get(estrategia, '#999')
        fig6.add_annotation(
            x=1.02, y=i,
            xref='paper', yref='y',
            text=f"<b>{estrategia}</b>",
            showarrow=False,
            font=dict(size=10, color='white'),
            bgcolor=color,
            borderpad=3
        )
    
    fig6.show()
else:
    print("No hay suficientes datos para el heatmap.")

### 7️⃣ Tabla Resumen Exportable
Genera una tabla consolidada con todos los parámetros de reabastecimiento lista para exportar.

In [229]:
# TABLA RESUMEN: Exportable para uso operativo
df_resumen = df_calc[['FAMILIA_LOGICA', 'CONSUMO_TOTAL', 'CONSUMO_NORMALIZADO', 'PROMEDIO_SEM', 'DESV_STD', 'CV', 
                       'PCT_SEMANAS_CONSUMO', 'LT_SEMANAS', 'CLASE_ABC', 
                       'CLASE_XYZ', 'ESTRATEGIA', 'POLITICA', 
                       'STOCK_SEGURIDAD', 'PUNTO_REORDEN']].copy()

# Renombrar columnas para mayor claridad
df_resumen.columns = ['Familia', 'Consumo Real', 'Score Importancia', 'Promedio Sem.', 'Desv. Std.', 'CV',
                       '% Sem. Activas', 'Lead Time', 'ABC', 
                       'XYZ', 'Estrategia', 'Política', 
                       'Stock Seg.', 'ROP']

# Formatear
df_resumen['Score Importancia'] = df_resumen['Score Importancia'].round(1)
df_resumen['% Sem. Activas'] = (df_resumen['% Sem. Activas'] * 100).round(1).astype(str) + '%'
df_resumen['Lead Time'] = df_resumen['Lead Time'].round(1)

# Ordenar por política y luego por Score
df_resumen = df_resumen.sort_values(['Política', 'ABC', 'Score Importancia'], 
                                      ascending=[True, True, False])

# Mostrar resumen
print("📋 TABLA RESUMEN DE PARÁMETROS DE REABASTECIMIENTO")
print("="*80)
print(f"Total de insumos: {len(df_resumen)}")
print(f"\n📊 DISTRIBUCIÓN POR POLÍTICA:")
for pol in ['ROP_AUTOMATICO', 'REVISION_PERIODICA', 'STOCK_MINIMO', 'BAJO_PEDIDO']:
    n = len(df_resumen[df_resumen['Política'] == pol])
    icon = {'ROP_AUTOMATICO':'🟢', 'REVISION_PERIODICA':'🟡', 'STOCK_MINIMO':'🟠', 'BAJO_PEDIDO':'⚪'}[pol]
    print(f"   {icon} {pol}: {n} items ({n/len(df_resumen)*100:.1f}%)")

# Mostrar tabla (primeros 20 registros)
print("\n" + "="*80)
print("📋 PRIMEROS 20 INSUMOS (ordenados por política y Score de Importancia):")
display(df_resumen.head(20))

📋 TABLA RESUMEN DE PARÁMETROS DE REABASTECIMIENTO
Total de insumos: 97

📊 DISTRIBUCIÓN POR POLÍTICA:
   🟢 ROP_AUTOMATICO: 15 items (15.5%)
   🟡 REVISION_PERIODICA: 13 items (13.4%)
   🟠 STOCK_MINIMO: 48 items (49.5%)
   ⚪ BAJO_PEDIDO: 21 items (21.6%)

📋 PRIMEROS 20 INSUMOS (ordenados por política y Score de Importancia):


,Familia,Consumo Real,Score Importancia,Promedio Sem.,Desv. Std.,CV,% Sem. Activas,Lead Time,ABC,XYZ,Estrategia,Política,Stock Seg.,ROP
INSUMO,,,,,,,,,,,,,,
CRP0010-CREMALLERA NYLON EKA 20CM,CREMALLERAS Y CIERRES,24.00,275.3,0.888889,4.618802,5.196152,3.7%,0.7,C,Z,CZ,BAJO_PEDIDO,0.0,1.0
BROCHE PLASTICO 11 MM,BOTONES Y HERRAJES,300.00,268.5,11.111111,57.735027,5.196152,3.7%,0.7,C,Z,CZ,BAJO_PEDIDO,0.0,8.0
HILADILLO 1CM,CINTAS Y CORDONES,36.90,256.5,1.366667,7.101408,5.196152,3.7%,0.7,C,Z,CZ,BAJO_PEDIDO,0.0,1.0
CREMALLERA NYLON 20CM,CREMALLERAS Y CIERRES,20.00,256.3,0.740741,3.849002,5.196152,3.7%,0.7,C,Z,CZ,BAJO_PEDIDO,0.0,1.0
HEBILLA PLASTICA,BOTONES Y HERRAJES,254.00,250.6,9.407407,48.882323,5.196152,3.7%,0.7,C,Z,CZ,BAJO_PEDIDO,0.0,7.0
"SEL0030 SESGO LICRADO 3,6 CM",CINTAS Y CORDONES,34.80,249.7,1.288889,6.697263,5.196152,3.7%,0.7,C,Z,CZ,BAJO_PEDIDO,0.0,1.0
CRN0003-CREMALLERA NYLON EKA 80CM #6,CREMALLERAS Y CIERRES,18.00,245.4,0.666667,3.464102,5.196152,3.7%,0.7,C,Z,CZ,BAJO_PEDIDO,0.0,0.0
COE0039-CORDON TUBULAR ESPECIAL,CINTAS Y CORDONES,32.50,241.8,1.203704,6.254628,5.196152,3.7%,0.7,C,Z,CZ,BAJO_PEDIDO,0.0,1.0
REC0057 RESORTE CAUCHO 4 CM,ELASTICOS,66.00,192.1,2.444444,12.701706,5.196152,3.7%,0.7,C,Z,CZ,BAJO_PEDIDO,0.0,2.0


In [230]:
df_resumen.sort_values(by='Score Importancia', ascending=False)

,Familia,Consumo Real,Score Importancia,Promedio Sem.,Desv. Std.,CV,% Sem. Activas,Lead Time,ABC,XYZ,Estrategia,Política,Stock Seg.,ROP
INSUMO,,,,,,,,,,,,,,
TAP0034 TANCA PUNTERA,ACCESORIOS PLASTICOS,555.00,1000.0,20.555556,51.995809,2.529526,37.0%,0.7,A,Y,AY,REVISION_PERIODICA,73.0,87.0
TMA0104-MALE,OTROS INSUMOS,22.00,1000.0,0.814815,4.233902,5.196152,3.7%,0.7,A,Z,AZ,STOCK_MINIMO,6.0,6.0
CRN0002-CREMALLERA NYLON YES 20CM,CREMALLERAS Y CIERRES,19267.00,1000.0,713.592593,968.578668,1.357327,92.6%,0.7,A,X,AX,ROP_AUTOMATICO,2861.0,3371.0
CUP0036-CUELLOS PUNOS,TEJIDOS (CUELLOS/PUÑOS),21450.00,1000.0,794.444444,1109.720697,1.396851,88.9%,2.1,A,X,AX,ROP_AUTOMATICO,7065.0,8768.0
GUA0027-GUATA ACOLCHADA 200,TEXTILES Y GUATAS,1719.20,1000.0,63.674074,137.519021,2.159733,63.0%,0.7,A,X,AX,ROP_AUTOMATICO,232.0,278.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GUA0027-GUATA ACOLCHADA 300,TEXTILES Y GUATAS,12.20,0.0,0.451852,2.347891,5.196152,3.7%,0.7,C,Z,CZ,BAJO_PEDIDO,0.0,0.0
FAJONES,TEJIDOS (CUELLOS/PUÑOS),4.90,0.0,0.181481,0.943005,5.196152,3.7%,2.1,C,Z,CZ,BAJO_PEDIDO,0.0,0.0
CRI0013-CREMALLERA NYLON YES 80CM,CREMALLERAS Y CIERRES,1.00,0.0,0.037037,0.192450,5.196152,3.7%,0.7,C,Z,CZ,BAJO_PEDIDO,0.0,0.0


In [231]:
# EXPORTAR A EXCEL (opcional)
# df_resumen.to_excel('reglas_reabastecimiento_insumos.xlsx', index=True)
# print("✅ Archivo exportado: reglas_reabastecimiento_insumos.xlsx")

In [ ]:
# GENERACIÓN DE REPORTE HTML ESTÁTICO
import datetime

# 1. Configuración del Reporte
titulo_reporte = "Reporte de Reglas de Reabastecimiento de Insumos"
fecha_generacion = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")

# 2. Explicación de la Lógica (HTML)
html_logica = """
<div style="font-family: Arial, sans-serif; margin: 20px;">
    <h2>1. Metodología y Rigurosidad Estadística</h2>
    <p>Este análisis utiliza un enfoque cuantitativo robusto para optimizar la gestión de inventarios, priorizando los recursos donde más impacto generan. La metodología se estructura en tres fases analíticas:</p>
    
    <h3>A. Normalización de Datos (El Desafío de las Unidades Heterogéneas)</h3>
    <p>Uno de los mayores retos en inventarios de confección es comparar insumos con unidades de medida dispares (ej: metros de tela vs. unidades de botones). Para resolver esto, no se usó el volumen bruto, sino un <b>Score de Importancia Relativa</b> calculado mediante el siguiente algoritmo:</p>
    <ol>
        <li><b>Agrupación por Familias Lógicas:</b> Se clasificaron los insumos en categorías comparables (ej: <i>Cremalleras</i>, <i>Textiles</i>, <i>Botones</i>).</li>
        <li><b>Transformación Logarítmica:</b> Se aplicó una escala logarítmica al consumo para suavizar las diferencias extremas de magnitud entre items masivos y unitarios.</li>
        <li><b>Escalamiento Intra-Familia (Min-Max):</b> Se asignó un puntaje de 0 a 1000 a cada item en función de su posición relativa dentro de su propia familia.
            <br><i>Ejemplo: El botón más consumido recibe 1000 puntos, igual que la tela más consumida, permitiendo que ambos compitan equitativamente por la atención en la clasificación ABC.</i>
        </li>
    </ol>

    <h3>B. Segmentación Multidimensional (Matriz ABC-XYZ)</h3>
    <p>Sobre la base del Score Normalizado, se aplicó una segmentación cruzada:</p>
    <ul>
        <li><b>Dimensión de Impacto (ABC - Pareto):</b> Se aplicó el principio de Pareto (80/20) sobre el Score acumulado.
            <ul>
                <li><b>Clase A (Críticos):</b> El 20% de los items que generan el 80% del impacto ponderado. Requieren control estricto.</li>
                <li><b>Clase B (Intermedios):</b> Importancia media.</li>
                <li><b>Clase C (Triviales):</b> La gran mayoría de items con bajo impacto relativo.</li>
            </ul>
        </li>
        <li><b>Dimensión de Variabilidad (XYZ - Frecuencia):</b> Se analizó la estabilidad de la demanda midiendo la frecuencia de consumo semanal.
            <ul>
                <li><b>Clase X (Estables):</b> Consumo constante (≥ 60% de las semanas). Altamente predecibles.</li>
                <li><b>Clase Y (Variables):</b> Consumo intermitente (30% - 60%).</li>
                <li><b>Clase Z (Esporádicos):</b> Consumo irregular (< 30%). Difíciles de pronosticar.</li>
            </ul>
        </li>
    </ul>

    <h3>C. Definición de Políticas y Nivel de Servicio</h3>
    <p>Para cada segmento se definió una estrategia matemáticamente óptima:</p>
    <ul>
        <li>🟢 <b>ROP Automático (AX, BX):</b> Para items estables e importantes. Se calculó un <b>Stock de Seguridad</b> basado en la desviación estándar de la demanda y la variabilidad del Lead Time, diseñado para garantizar un <b>Nivel de Servicio del 95%</b>.</li>
        <li>🟡 <b>Revisión Periódica (AY, BY):</b> Para items de impacto medio/alto pero demanda variable. Se sugiere revisar niveles en intervalos fijos.</li>
        <li>⚪ <b>Bajo Pedido / Stock Mínimo (Z):</b> Para items erráticos donde los modelos estadísticos pierden confiabilidad, se opta por coberturas mínimas o compra contra orden.</li>
    </ul>

    <hr style="border: 0; border-top: 1px solid #eee; margin: 20px 0;">

    <h2>2. Guía de Interpretación de Resultados (Uso Práctico)</h2>
    <p>La tabla final de este reporte es la herramienta operativa para la toma de decisiones. A continuación se detalla cómo interpretar sus columnas clave:</p>
    
    <div style="background-color: #f8f9fa; padding: 15px; border-left: 4px solid #3498db; border-radius: 4px;">
        <ul style="margin: 0; padding-left: 20px;">
            <li style="margin-bottom: 10px;"><b>Score Importancia:</b> Es el valor normalizado (0-1000). Indica qué tan "líder" es el insumo dentro de su familia.</li>
            <li style="margin-bottom: 10px;"><b>% Sem. Activas:</b> Mide la regularidad. <i>(100% = uso semanal continuo; 10% = uso muy esporádico).</i></li>
            <li style="margin-bottom: 10px;"><b>Política Sugerida:</b> La acción recomendada.
                <ul>
                    <li><i>ROP Automático:</i> Configurar en el ERP. El sistema debe disparar compra automáticamente.</li>
                    <li><i>Bajo Pedido:</i> No mantener stock. Comprar solo contra pedido de cliente confirmado.</li>
                </ul>
            </li>
            <li style="margin-bottom: 10px;"><b>ROP (Punto de Reorden):</b> <span style="color: #e74c3c; font-weight: bold;">EL DATO CRÍTICO.</span> Es el nivel de inventario que dispara la reposición.
                <br><i>Interpretación: "Cuando el inventario físico caiga por debajo de [Valor ROP], se debe generar una orden de compra inmediatamente".</i></li>
            <li><b>Stock Seg. (Seguridad):</b> Es el "colchón" calculado para absorber imprevistos. Ya está incluido dentro del cálculo del ROP.</li>
        </ul>
    </div>
</div>
"""

# 3. Convertir Gráficos a HTML
html_fig5 = fig5.to_html(full_html=False, include_plotlyjs='cdn')
html_fig1 = fig1.to_html(full_html=False, include_plotlyjs=False)
html_fig2 = fig2.to_html(full_html=False, include_plotlyjs=False)
html_fig3 = fig3.to_html(full_html=False, include_plotlyjs=False)
html_fig4 = fig4.to_html(full_html=False, include_plotlyjs=False)
html_fig6 = fig6.to_html(full_html=False, include_plotlyjs=False)

# 4. Tabla Resumen con Buscador Inteligente
# Preparar DataFrame con columna Insumo explícita
df_tabla = df_resumen.reset_index()
# Renombrar columna de índice si es necesario (usualmente es 'INSUMO' o 'index')
col_rename = {df_tabla.columns[0]: 'Insumo'} 
df_tabla.rename(columns=col_rename, inplace=True)

# Generar opciones para el datalist (Lista desplegable de búsqueda)
opciones_insumos = "".join([f"<option value='{x}'>" for x in df_tabla['Insumo'].astype(str).unique()])

# Estilos CSS
estilo_tabla = """
<style>
    .table-container {
        max-height: 600px;
        overflow-y: auto;
        border: 1px solid #ddd;
        margin-top: 10px;
        box-shadow: 0 4px 8px rgba(0,0,0,0.1);
    }
    table {
        width: 100%;
        border-collapse: collapse;
        font-family: Arial, sans-serif;
        font-size: 12px;
    }
    th {
        background-color: #2c3e50;
        color: white;
        position: sticky;
        top: 0;
        padding: 10px;
        text-align: left;
        z-index: 10;
    }
    td {
        padding: 8px;
        border-bottom: 1px solid #eee;
    }
    tr:nth-child(even) {
        background-color: #f9f9f9;
    }
    tr:hover {
        background-color: #e8f6f3;
    }
    /* Estilo para el buscador */
    .search-box {
        background: #f8f9fa;
        padding: 15px;
        border-radius: 8px;
        border: 1px solid #e9ecef;
        margin-bottom: 15px;
        display: flex;
        align-items: center;
        gap: 10px;
    }
    .search-input {
        padding: 10px;
        width: 100%;
        max-width: 400px;
        border: 1px solid #ced4da;
        border-radius: 4px;
        font-size: 14px;
    }
</style>
"""

# Script JS para filtrado
script_buscador = """
<script>
function filterTable() {
  var input, filter, table, tr, td, i, txtValue;
  input = document.getElementById("insumoInput");
  filter = input.value.toUpperCase();
  table = document.querySelector("table.dataframe");
  tr = table.getElementsByTagName("tr");
  
  // Recorrer todas las filas de la tabla (excepto el encabezado)
  for (i = 1; i < tr.length; i++) {
    // Columna 0 es 'Insumo'
    td = tr[i].getElementsByTagName("td")[0];
    if (td) {
      txtValue = td.textContent || td.innerText;
      if (txtValue.toUpperCase().indexOf(filter) > -1) {
        tr[i].style.display = "";
      } else {
        tr[i].style.display = "none";
      }
    }       
  }
}
</script>
"""

html_tabla = df_tabla.to_html(index=False, border=0, classes="dataframe")

html_tabla_container = f"""
<div style="font-family: Arial, sans-serif; margin: 20px;">
    <h2>4. Tabla Resumen de Parámetros (Detalle Completo)</h2>
    <p>Utilice el buscador para encontrar rápidamente un insumo específico o filtrar por familia.</p>
    
    {estilo_tabla}
    
    <div class="search-box">
        <label for="insumoInput" style="font-weight:bold; color:#2c3e50;">🔍 Buscar Insumo:</label>
        <input list="insumos_list" id="insumoInput" class="search-input" onkeyup="filterTable()" placeholder="Escriba el nombre del insumo...">
        <datalist id="insumos_list">
            {opciones_insumos}
        </datalist>
        <span style="color: #7f8c8d; font-size: 0.9em;">(Seleccione de la lista o escriba para filtrar)</span>
    </div>

    <div class="table-container">
        {html_tabla}
    </div>
    
    {script_buscador}
</div>
"""

# 5. Ensamblar el HTML Final
html_completo = f"""
<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8">
    <title>{titulo_reporte}</title>
    <style>
        body {{ font-family: Arial, sans-serif; margin: 0; padding: 20px; background-color: #f4f6f7; }}
        .container {{ max-width: 1200px; margin: 0 auto; background-color: white; padding: 30px; border-radius: 8px; box-shadow: 0 0 15px rgba(0,0,0,0.1); }}
        h1 {{ color: #2c3e50; text-align: center; border-bottom: 2px solid #3498db; padding-bottom: 10px; }}
        .footer {{ text-align: center; margin-top: 30px; color: #7f8c8d; font-size: 12px; }}
        .chart-container {{ margin-bottom: 40px; border: 1px solid #eee; padding: 10px; border-radius: 5px; }}
        h2 {{ color: #34495e; border-left: 5px solid #3498db; padding-left: 10px; margin-top: 30px; }}
    </style>
</head>
<body>
    <div class="container">
        <h1>{titulo_reporte}</h1>
        <p style="text-align: center; color: #7f8c8d;">Generado el: {fecha_generacion}</p>
        
        {html_logica}
        
        <div style="font-family: Arial, sans-serif; margin: 20px;">
            <h2>3. Visualización de Resultados</h2>
        </div>

        <div class="chart-container">
            <h3>Resumen Ejecutivo</h3>
            {html_fig5}
        </div>
        
        <div class="chart-container">
            <h3>Matriz Estratégica ABC-XYZ</h3>
            {html_fig1}
        </div>

        <div class="chart-container">
            <h3>Análisis de Pareto (Score de Importancia)</h3>
            {html_fig2}
        </div>

        <div class="chart-container">
            <h3>Mapa de Dispersión: Frecuencia vs Volumen</h3>
            {html_fig3}
        </div>

        <div class="chart-container">
            <h3>Top Insumos con ROP (Punto de Reorden)</h3>
            {html_fig4}
        </div>

        <div class="chart-container">
            <h3>Patrón de Consumo Semanal (Heatmap)</h3>
            {html_fig6}
        </div>

        {html_tabla_container}
        
        <div class="footer">
            <p>Reporte generado por Mateo Herrera Muñoz.</p>
        </div>
    </div>
</body>
</html>
"""

# 6. Guardar archivo
nombre_archivo = "reporte_reabastecimiento.html"
with open(nombre_archivo, "w", encoding="utf-8") as f:
    f.write(html_completo)

print(f"✅ Reporte generado exitosamente: {nombre_archivo}")
from IPython.display import FileLink
display(FileLink(nombre_archivo))

✅ Reporte generado exitosamente: reporte_reabastecimiento.html


c:\Users\Analitycs Data\OneDrive - MOLT\Documentos\MOLT\Analytics\Produccion\reporte_reabastecimiento.html